In [2]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from math import factorial, tan, pi

# The general form of mathematically representing an airfoil:


\begin{equation}
    \psi = \frac{x}{c} \; and \;
    \zeta = \frac{z}{c} \; and \;
    \zeta_T = \frac{\Delta\zeta_{TE}}{c} 
\end{equation}

The general mathematical representation of an airfoil using this method is :

\begin{equation}
    \zeta(\psi) = \sqrt{\psi} * (1 - \psi) * \Sigma_{i = 0}^{N}{(A_i* \psi^i) + \psi * \zeta_T}
\end{equation}

(1) the term $\sqrt{\psi}$ esures that the leading edge is round. and the term $(1 - \psi)$ ensures a sharp trailing edge.

(2) the term $\psi * \zeta_T$ controls the trailing edge thickness.

(3) the most important term all in all is of course $\Sigma_{i = 0}^{N}{(A_i* \psi^i)}$ ; which is the function that represents the shape between leading and trailing edge. This equation can be any analytic function.



# The airfoil shape function:

Defined shape function is as follows:

$$ S(\psi) = \frac{\zeta(\psi) - \psi * \zeta_T}{\sqrt{\psi}*(1-\psi)} \; $$

which is actually the representation of (3) in other words:

$$ S(\frac{x}{L}) = \Sigma_{i = 0}^{N}{(A_i* [\frac{x}{L}]^i)} $$

two boundary conditions to be fixed for each airfoil are as follows:

$$S(0) = \sqrt{2\frac{\color{red}{R_{LE}}}{c}}$$
$$ S(1) = tan(\color{red}\beta) + \frac{\color{red}{\Delta Z_{TE}}}{c}$$

The class function is the modification of the denominator of shape function:
$$ C_{\color{red}{N2}}^{\color{red}{N1}}(\psi) = (\sqrt{\psi})^{N1}*(1-\psi)^{N2} : Class function$$

the parameters N1 and N2 of the class function define the type of the airfoil that will be created.

The “class function” is used to define general classes of geometries, where as the “shape function” is used to define specific shapes within the geometry class. 

$$ \zeta(\psi) = C_{N2}^{N1}(\psi) * S(\psi)  + \psi * \zeta_T$$


In [41]:
def binomial(r, n):
    return factorial(n) / (factorial(r) * factorial(n-r))

def bpn(x, r, n):
    x = np.array(x)
    results = binomial(r,n) * (x**r) * ((1-x)**(n-r))
    return results
def show_bpn(x, n):
    for r in range(n+1):
        plt.plot(x, bpn(x, r, n))
    plt.show()
    return
def get_class(x, n1, n2):
    return (x ** n1) * ((1-x)**n2) 
def show_af_bpn(x, n1, n2, n):
    c = get_class(x, n1, n2)
    for r in range(n+1):
        plt.plot(x, bpn(x, r, n) * c)
    plt.show()
    return

def airfoil(Nnose = 0.5,
            Naft  = 1.0,
            n     = 4,
            x     = np.concatenate((np.linspace(0, 0.195, 81), np.linspace(0.2, 1., 101))) ,
            rle   = 0.01,
            bu    = 20.,
            dzu   = 0.002,
            bl    = 20.,
            dzl   = -0.001,
            au_lim = [-0.1, 0.6],
            al_lim = [-0.6, 0.1], show=True):
    
    au    = np.random.uniform(au_lim[0], au_lim[1], n+1)
    al    = np.random.uniform(al_lim[0], al_lim[1], n+1)
    au[0] = (rle *2)**0.5
    au[-1]= tan(bu * pi / 180.) + dzu
    al[0] = -(rle *2)**0.5
    al[-1]= tan(bl * pi / 180.) + dzl

    # the calculation
    c = get_class(x, Nnose, Naft)
    # create upper surface
    # calculate Si
    resu = np.zeros_like(x)
    resl = np.zeros_like(x)

    for i in range(0, n+1):
        resu += au[i] * bpn(x, i, n)
        resl += al[i] * bpn(x, i, n)

    resu = resu * c + x * dzu
    resl = resl * c + x * dzl
    
    data_structure = {}
    data_structure['x'] = x
    data_structure['yu'] = resu
    data_structure['yl'] = resl
    data_structure['au'] = au
    data_structure['al'] = al
    data_structure['Nnose'] = Nnose
    data_structure['Naft'] = Naft
    data_structure['rle'] = rle
    data_structure['b'] = [bu, bl]
    data_structure['dz'] = [dzu, dzl]
    data_structure['n'] = n
    if show:
        fig, ax = plt.subplots()
        ax.plot(x, resu)
        ax.plot(x, resl)
        ax.axis('equal')
        fig.show()
    return data_structure

In [43]:
af = []
for i in range(1000):
    n=np.random.randint(2,8)
    rle=np.random.uniform(0.01,0.05)
    bu=np.random.randint(-10,20)
    bl=np.random.randint(-20,bu)
    dzu = np.random.uniform(-0.001, 0.005)
    dzl = np.random.uniform(-0.005, dzu-0.001)
    Nnose = np.random.uniform(0.45,0.55)
    Naft  = np.random.uniform(0.95, 1.0)
    
    af.append(airfoil(show=False, n=n, rle=rle, bu=bu, bl=bl, dzl=dzl, dzu=dzu, Naft=Naft, Nnose=Nnose, 
                      x=np.concatenate((np.linspace(0, 0.195, 61), np.linspace(0.2, 1., 101)))))

In [1]:
%matplotlib
# parameters
#seed = 0
#np.random.seed = seed
Nnose = 0.5
Naft  = 1.0
n     = 4
x     = np.concatenate((np.linspace(0, 0.095, 51), np.linspace(0.1, 1., 101))) 
rle   = 0.01
bu    = 20.
dzu   = 0.002
au    = np.random.uniform(-0.1, 0.5, n+1)
au[0] = (rle *2)**0.5
au[-1]= tan(bu * pi / 180.) + dzu
bl    = 20.
dzl   = -0.001
al    = np.random.uniform(-0.5, 0.1, n+1)
al[0] = -(rle *2)**0.5
al[-1]= tan(bl * pi / 180.) + dzl

# the calculation
c = get_class(x, Nnose, Naft)
# create upper surface
# calculate Si
resu = np.zeros_like(x)
resl = np.zeros_like(x)

for i in range(0, n+1):
    resu += au[i] * bpn(x, i, n)
    resl += al[i] * bpn(x, i, n)
    
resu = resu * c + x * dzu
resl = resl * c + x * dzl
fig, ax = plt.subplots()
ax.plot(x, resu)
ax.plot(x, resl)
ax.axis('equal')
fig.show()

Using matplotlib backend: Qt5Agg


NameError: name 'np' is not defined